# BONE AGE PREDICTION - GPU OPTIMIZED (T4 x2)
## Multi-Task Learning: Regression + Classification
### Dataset: RSNA Bone Age (kmader/rsna-bone-age)
### Hardware: 2x NVIDIA Tesla T4 GPUs

In [11]:
# ============================================================================
# IMPORTS & SETUP
# ============================================================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, classification_report, confusion_matrix

from tensorflow.keras import layers, models, optimizers, losses, metrics, callbacks
from tensorflow.keras.applications import InceptionV3

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow Version: 2.20.0
GPU Available: []


In [12]:
# ============================================================================
# GPU STRATEGY CONFIGURATION (Multi-GPU Support)
# ============================================================================
print("\n>>> CONFIGURING GPU STRATEGY")

# Detect available GPUs and create appropriate strategy
gpus = tf.config.list_physical_devices('GPU')
print(f"Number of GPUs detected: {len(gpus)}")

if len(gpus) > 1:
    # Use MirroredStrategy for multi-GPU training
    strategy = tf.distribute.MirroredStrategy()
    print(f"Using MirroredStrategy with {strategy.num_replicas_in_sync} GPUs")
elif len(gpus) == 1:
    # Single GPU - use default strategy
    strategy = tf.distribute.get_strategy()
    print("Using single GPU")
else:
    # CPU fallback
    strategy = tf.distribute.get_strategy()
    print("No GPU found, using CPU")

# Enable mixed precision for faster training on T4 GPUs
#tf.keras.mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision DISABLED (to prevent NaN losses)")



>>> CONFIGURING GPU STRATEGY
Number of GPUs detected: 0
No GPU found, using CPU
Mixed precision DISABLED (to prevent NaN losses)


In [13]:
# ============================================================================
# HYPERPARAMETERS & CONFIGURATION
# ============================================================================
# Scale batch size by number of GPUs for optimal throughput
BATCH_SIZE_PER_REPLICA = 16  # Per GPU batch size
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

IMG_SIZE = 299  # InceptionV3 native input size
EPOCHS = 50     # Will use early stopping
LEARNING_RATE = 0.0001

# Dataset paths for Kaggle
# BASE_DIR = '/kaggle/input/rsna-bone-age' 
BASE_DIR = r'D:\PRML\PRML_PROJECT\archive'
TRAIN_IMG_DIR = os.path.join(BASE_DIR, 'boneage-training-dataset', 'boneage-training-dataset')
TRAIN_CSV_PATH = os.path.join(BASE_DIR, 'boneage-training-dataset.csv')

# Verify paths exist
print(f"\nDataset Configuration:")
print(f"Base Directory: {BASE_DIR}")
print(f"Images Directory: {TRAIN_IMG_DIR}")
print(f"CSV Path: {TRAIN_CSV_PATH}")
print(f"\nBatch Size: {BATCH_SIZE} (per replica: {BATCH_SIZE_PER_REPLICA})")
print(f"Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Max Epochs: {EPOCHS}")


Dataset Configuration:
Base Directory: D:\PRML\PRML_PROJECT\archive
Images Directory: D:\PRML\PRML_PROJECT\archive\boneage-training-dataset\boneage-training-dataset
CSV Path: D:\PRML\PRML_PROJECT\archive\boneage-training-dataset.csv

Batch Size: 16 (per replica: 16)
Image Size: 299x299
Max Epochs: 50


In [14]:
# ============================================================================
# DATA LOADING & PREPROCESSING
# ============================================================================
print("\n>>> LOADING AND PREPARING DATA")

# Load CSV metadata
df = pd.read_csv(TRAIN_CSV_PATH)
print(f"Total samples loaded: {len(df)}")
print(f"\nDataFrame columns: {df.columns.tolist()}")
print(f"\nFirst few rows:\n{df.head()}")

# Create full image paths
df['path'] = df['id'].apply(lambda x: os.path.join(TRAIN_IMG_DIR, f"{x}.png"))

# Convert gender to float for model input
df['gender_float'] = df['male'].astype('float32')

# Verify a few image paths exist
sample_paths = df['path'].head(3).tolist()
print(f"\nVerifying sample image paths:")
for p in sample_paths:
    exists = os.path.exists(p)
    print(f"  {os.path.basename(p)}: {'✓' if exists else '✗'}")

# ============================================================================
# AGE DISCRETIZATION FOR CLASSIFICATION TASK
# ============================================================================
# Define age buckets (in months):
# 0: Infant (0-24 months / 0-2 years)
# 1: Pre-Puberty (25-120 months / 2-10 years)
# 2: Puberty (121-192 months / 10-16 years)
# 3: Young Adult (193+ months / 16+ years)

def discretize_age(age_months):
    if age_months <= 24:
        return 0
    elif age_months <= 120:
        return 1
    elif age_months <= 192:
        return 2
    else:
        return 3

df['age_class'] = df['boneage'].apply(discretize_age)
NUM_CLASSES = 4
CLASS_NAMES = ['Infant (0-2y)', 'Pre-Puberty (2-10y)', 'Puberty (10-16y)', 'Young Adult (16+y)']

print(f"\n>>> AGE STATISTICS")
print(f"Age range: {df['boneage'].min():.1f} - {df['boneage'].max():.1f} months")
print(f"Age mean: {df['boneage'].mean():.1f} months")
print(f"Age std: {df['boneage'].std():.1f} months")

print(f"\n>>> CLASS DISTRIBUTION")
class_dist = df['age_class'].value_counts().sort_index()
for idx, count in class_dist.items():
    print(f"  Class {idx} ({CLASS_NAMES[idx]}): {count} samples ({count/len(df)*100:.1f}%)")


>>> LOADING AND PREPARING DATA
Total samples loaded: 12611

DataFrame columns: ['id', 'boneage', 'male']

First few rows:
     id  boneage   male
0  1377      180  False
1  1378       12  False
2  1379       94  False
3  1380      120   True
4  1381       82  False

Verifying sample image paths:
  1377.png: ✗
  1378.png: ✗
  1379.png: ✗

>>> AGE STATISTICS
Age range: 1.0 - 228.0 months
Age mean: 127.3 months
Age std: 41.2 months

>>> CLASS DISTRIBUTION
  Class 0 (Infant (0-2y)): 168 samples (1.3%)
  Class 1 (Pre-Puberty (2-10y)): 5112 samples (40.5%)
  Class 2 (Puberty (10-16y)): 6985 samples (55.4%)
  Class 3 (Young Adult (16+y)): 346 samples (2.7%)


In [15]:
# ============================================================================
# TRAIN/VAL/TEST SPLIT (70/15/15)
# ============================================================================
print("\n>>> SPLITTING DATA")

# Stratified split to maintain class distribution
train_df, temp_df = train_test_split(
    df, 
    test_size=0.3, 
    random_state=42, 
    stratify=df['age_class']
)

val_df, test_df = train_test_split(
    temp_df, 
    test_size=0.5, 
    random_state=42, 
    stratify=temp_df['age_class']
)

print(f"Train set: {len(train_df)} samples ({len(train_df)/len(df)*100:.1f}%)")
print(f"Val set: {len(val_df)} samples ({len(val_df)/len(df)*100:.1f}%)")
print(f"Test set: {len(test_df)} samples ({len(test_df)/len(df)*100:.1f}%)")

# Reset indices for clean iteration
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


>>> SPLITTING DATA
Train set: 8827 samples (70.0%)
Val set: 1892 samples (15.0%)
Test set: 1892 samples (15.0%)


In [16]:
# ============================================================================
# TF.DATA PIPELINE (Optimized for GPU)
# ============================================================================
print("\n>>> BUILDING DATA PIPELINES")

def load_and_preprocess_image(path):
    """Load image, decode, resize, and normalize for InceptionV3."""
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    # InceptionV3 preprocessing: scale to [-1, 1]
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    return image

def create_dataset(dataframe, is_training=True, augment=True):
    """
    Create optimized tf.data.Dataset for multi-input/multi-output model.
    
    Inputs: (image, gender)
    Outputs: {'age_out': boneage, 'class_out': age_class}
    """
    # Create dataset from paths and gender
    paths = dataframe['path'].values
    genders = dataframe['gender_float'].values
    ages = dataframe['boneage'].values.astype('float32')
    classes = dataframe['age_class'].values.astype('int32')
    
    # Build dataset
    dataset = tf.data.Dataset.from_tensor_slices((
        {'paths': paths, 'genders': genders},
        {'age_out': ages, 'class_out': classes}
    ))
    
    # Shuffle before loading images (more efficient)
    if is_training:
        dataset = dataset.shuffle(buffer_size=2000, seed=42)
    
    # Load and preprocess images
    def process_data(inputs, labels):
        image = load_and_preprocess_image(inputs['paths'])
        return (image, inputs['genders']), labels
    
    dataset = dataset.map(process_data, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Data augmentation for training only
    if is_training and augment:
        def augment_image(inputs, labels):
            image, gender = inputs
            # Random horizontal flip (X-rays can be mirrored)
            image = tf.image.random_flip_left_right(image)
            # Slight rotation
            image = tf.image.rot90(image, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
            # Random brightness/contrast
            image = tf.image.random_brightness(image, max_delta=0.1)
            image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
            # Clip to valid range
            image = tf.clip_by_value(image, -1.0, 1.0)
            return (image, gender), labels
        
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Batch and prefetch
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=is_training)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return dataset

# Create datasets
train_ds = create_dataset(train_df, is_training=True, augment=True)
val_ds = create_dataset(val_df, is_training=False, augment=False)
test_ds = create_dataset(test_df, is_training=False, augment=False)

print("✓ Data pipelines created successfully")
print(f"  Train batches: ~{len(train_df) // BATCH_SIZE}")
print(f"  Val batches: ~{len(val_df) // BATCH_SIZE}")
print(f"  Test batches: ~{len(test_df) // BATCH_SIZE}")


>>> BUILDING DATA PIPELINES
✓ Data pipelines created successfully
  Train batches: ~551
  Val batches: ~118
  Test batches: ~118


In [17]:
# ============================================================================
# MODEL BUILDING (Multi-Input/Multi-Output CNN)
# ============================================================================
print("\n>>> BUILDING MODEL")

# Build model within strategy scope for multi-GPU support
with strategy.scope():
    # ========== INPUTS ==========
    image_input = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='image_input')
    gender_input = layers.Input(shape=(1,), name='gender_input')
    
    # ========== BACKBONE (InceptionV3) ==========
    # Load pretrained InceptionV3 without top layers
    base_model = InceptionV3(
        include_top=False,
        weights='imagenet',  # Will download if internet enabled, or use cached
        input_tensor=image_input,
        pooling=None
    )
    
    # Fine-tune the entire model
    base_model.trainable = True
    
    # Optionally freeze early layers for faster training
    # Uncomment to freeze first 200 layers:
    # for layer in base_model.layers[:200]:
    #     layer.trainable = False
    
    print(f"Base model: {base_model.name}")
    print(f"Total layers: {len(base_model.layers)}")
    print(f"Trainable: {base_model.trainable}")
    
    # ========== FEATURE EXTRACTION ==========
    x = base_model.output
    x = layers.GlobalAveragePooling2D(name='global_pool')(x)
    x = layers.Dropout(0.5, name='dropout_features')(x)
    
    # ========== COMBINE WITH GENDER ==========
    # Concatenate image features with gender information
    combined = layers.Concatenate(name='concat_features')([x, gender_input])
    
    # Shared dense layer
    z = layers.Dense(256, activation='relu', name='dense_shared')(combined)
    z = layers.Dropout(0.3, name='dropout_shared')(z)
    
    # ========== OUTPUT HEADS ==========
    # Regression head: Predict continuous age in months
    age_output = layers.Dense(1, activation='linear', dtype='float32', name='age_out')(z)
    
    # Classification head: Predict age category
    class_output = layers.Dense(NUM_CLASSES, activation='softmax', dtype='float32', name='class_out')(z)
    
    # ========== ASSEMBLE MODEL ==========
    model = models.Model(
        inputs=[image_input, gender_input],
        outputs=[age_output, class_output],
        name='BoneAgeMultiTask'
    )
    
    # ========== COMPILE MODEL ==========
    optimizer = optimizers.Adam(learning_rate=LEARNING_RATE)
    
    # Define losses for each output
    losses_dict = {
        'age_out': 'mae',  # Mean Absolute Error for regression
        'class_out': 'sparse_categorical_crossentropy'  # For integer labels
    }
    
    # Weight the losses (prioritize regression if needed)
    loss_weights = {
        'age_out': 2.0,  # Regression is primary task
        'class_out': 1.0
    }
    
    # Define metrics for each output
    metrics_dict = {
        'age_out': ['mae', 'mse'],
        'class_out': ['accuracy']
    }
    
    model.compile(
        optimizer=optimizer,
        loss=losses_dict,
        loss_weights=loss_weights,
        metrics=metrics_dict
    )

print("\n✓ Model compiled successfully")
print(f"\nModel Summary:")
model.summary()

# Calculate total parameters
total_params = model.count_params()
print(f"\nTotal parameters: {total_params:,}")


>>> BUILDING MODEL
Base model: inception_v3
Total layers: 311
Trainable: True

✓ Model compiled successfully

Model Summary:


Model: "BoneAgeMultiTask"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)      │ (None, 299, 299, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_94 (Conv2D)            │ (None, 149, 149, 32)      │             864 │ image_input[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_94        │ (None, 149, 149, 32)      │              96 │ conv2d_94[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_94 (Activation)    │ (None, 149, 149, 32)      │               0 │ batch_normalization_94[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_95 (Conv2D)            │ (None, 147, 147, 32)      │           9,216 │ activation_94[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_95        │ (None, 147, 147, 32)      │              96 │ conv2d_95[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_95 (Activation)    │ (None, 147, 147, 32)      │               0 │ batch_normalization_95[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_96 (Conv2D)            │ (None, 147, 147, 64)      │          18,432 │ activation_95[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_96        │ (None, 147, 147, 64)      │             192 │ conv2d_96[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_96 (Activation)    │ (None, 147, 147, 64)      │               0 │ batch_normalization_96[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_4               │ (None, 73, 73, 64)        │               0 │ activation_96[0][0]        │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_97 (Conv2D)            │ (None, 73, 73, 80)        │           5,120 │ max_pooling2d_4[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_97        │ (None, 73, 73, 80)        │             240 │ conv2d_97[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_97 (Activation)    │ (None, 73, 73, 80)        │               0 │ batch_normalization_97[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_98 (Conv2D)            │ (None, 71, 71, 192)       │         138,24

 Total params: 22,328,869 (85.18 MB)

 Trainable params: 22,294,437 (85.05 MB)

 Non-trainable params: 34,432 (134.50 KB)


Total parameters: 22,328,869


In [18]:
# ============================================================================
# CALLBACKS FOR TRAINING
# ============================================================================
print("\n>>> SETTING UP CALLBACKS")

# Model checkpoint - save best model based on validation MAE
checkpoint_cb = callbacks.ModelCheckpoint(
    filepath='best_bone_age_model.keras',
    monitor='val_age_out_mae',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Early stopping - stop if no improvement
early_stop_cb = callbacks.EarlyStopping(
    monitor='val_age_out_mae',
    patience=8,
    mode='min',  # ← THIS LINE WAS ADDED
    restore_best_weights=True,
    verbose=1
)

# Reduce learning rate on plateau
reduce_lr_cb = callbacks.ReduceLROnPlateau(
    monitor='val_age_out_mae',
    factor=0.5,
    patience=4,
    min_lr=1e-7,
    verbose=1
)

# CSV logger for training history
csv_logger_cb = callbacks.CSVLogger('training_history.csv')

callback_list = [
    checkpoint_cb,
    early_stop_cb,
    reduce_lr_cb,
    csv_logger_cb
]

print("✓ Callbacks configured")


>>> SETTING UP CALLBACKS
✓ Callbacks configured


In [19]:
# # ============================================================================
# # TRAINING
# # ============================================================================
# print(f"\n{'='*80}")
# print(f"STARTING TRAINING")
# print(f"{'='*80}")
# print(f"Max Epochs: {EPOCHS}")
# print(f"Batch Size: {BATCH_SIZE}")
# print(f"Learning Rate: {LEARNING_RATE}")
# print(f"Strategy: {strategy.__class__.__name__}")
# print(f"GPUs: {strategy.num_replicas_in_sync}")
# print(f"{'='*80}\n")

# history = model.fit(
#     train_ds,
#     epochs=EPOCHS,
#     validation_data=val_ds,
#     callbacks=callback_list,
#     verbose=1
# )

# print(f"\n{'='*80}")
# print("TRAINING COMPLETE")
# print(f"{'='*80}")

In [20]:
# ============================================================================
# EVALUATION ON TEST SET
# ============================================================================
print("\n>>> EVALUATING ON TEST SET")

# Load best weights
model.load_weights('best_bone_age_model.keras')
print("✓ Loaded best model weights")

# Evaluate
test_results = model.evaluate(test_ds, verbose=1)

# Parse results
# Results format: [total_loss, age_loss, class_loss, age_mae, age_mse, class_accuracy]
print(f"\n{'='*80}")
print("TEST SET RESULTS")
print(f"{'='*80}")
print(f"Total Loss: {test_results[0]:.4f}")
print(f"\nRegression (Age Prediction):")
print(f"  MAE: {test_results[3]:.2f} months")
print(f"  MSE: {test_results[4]:.2f}")
print(f"  RMSE: {np.sqrt(test_results[4]):.2f} months")
print(f"\nClassification (Age Category):")
print(f"  Accuracy: {test_results[5]*100:.2f}%")
print(f"{'='*80}")


>>> EVALUATING ON TEST SET


C:\Users\yalam\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 390 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


✓ Loaded best model weights


NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
NewRandomAccessFile failed to Create/Open: D:\PRML\PRML_PROJECT\archive\boneage-training-dataset\boneage-training-dataset\6487.png : The system cannot find the path specified.
; No such process
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_20006]

In [ ]:
# ============================================================================
# GENERATE PREDICTIONS FOR DETAILED ANALYSIS
# ============================================================================
print("\n>>> GENERATING PREDICTIONS FOR ANALYSIS")

# Collect all test data
test_images_list = []
test_genders_list = []
test_ages_list = []
test_classes_list = []

for (images, genders), labels in test_ds:
    test_images_list.append(images)
    test_genders_list.append(genders)
    test_ages_list.append(labels['age_out'])
    test_classes_list.append(labels['class_out'])

# Concatenate batches
test_images_all = tf.concat(test_images_list, axis=0)
test_genders_all = tf.concat(test_genders_list, axis=0)
y_true_age = tf.concat(test_ages_list, axis=0).numpy()
y_true_class = tf.concat(test_classes_list, axis=0).numpy()

# Make predictions
predictions = model.predict([test_images_all, test_genders_all], verbose=1)
y_pred_age = predictions[0].flatten()
y_pred_class_probs = predictions[1]
y_pred_class = np.argmax(y_pred_class_probs, axis=1)

print(f"✓ Generated predictions for {len(y_true_age)} test samples")

In [ ]:
# ============================================================================
# VISUALIZATION: TRAINING HISTORY
# ============================================================================
print("\n>>> PLOTTING TRAINING HISTORY")

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Regression MAE
axes[0, 0].plot(history.history['age_out_mae'], label='Train MAE', linewidth=2)
axes[0, 0].plot(history.history['val_age_out_mae'], label='Val MAE', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('MAE (months)', fontsize=12)
axes[0, 0].set_title('Regression: Mean Absolute Error', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Classification Accuracy
axes[0, 1].plot(history.history['class_out_accuracy'], label='Train Accuracy', linewidth=2)
axes[0, 1].plot(history.history['val_class_out_accuracy'], label='Val Accuracy', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Accuracy', fontsize=12)
axes[0, 1].set_title('Classification: Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Total Loss
axes[1, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[1, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Loss', fontsize=12)
axes[1, 0].set_title('Total Loss (Weighted)', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Learning Rate (if available)
if 'lr' in history.history:
    axes[1, 1].plot(history.history['lr'], linewidth=2, color='orange')
    axes[1, 1].set_xlabel('Epoch', fontsize=12)
    axes[1, 1].set_ylabel('Learning Rate', fontsize=12)
    axes[1, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 1].set_yscale('log')
    axes[1, 1].grid(True, alpha=0.3)
else:
    axes[1, 1].text(0.5, 0.5, 'Learning Rate\nNot Logged', 
                    ha='center', va='center', fontsize=14)
    axes[1, 1].axis('off')

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Training history plot saved as 'training_history.png'")

In [ ]:
# ============================================================================
# VISUALIZATION: REGRESSION ANALYSIS
# ============================================================================
print("\n>>> PLOTTING REGRESSION RESULTS")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Predicted vs Actual (colored by gender)
gender_colors = test_genders_all.numpy().flatten()
scatter = axes[0].scatter(y_true_age, y_pred_age, 
                         c=gender_colors, cmap='coolwarm', 
                         alpha=0.5, s=30, edgecolors='black', linewidth=0.5)
axes[0].plot([0, 230], [0, 230], 'k--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Bone Age (months)', fontsize=13)
axes[0].set_ylabel('Predicted Bone Age (months)', fontsize=13)
axes[0].set_title(f'Regression: Predicted vs Actual\nTest MAE: {test_results[3]:.2f} months', 
                 fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
cbar = plt.colorbar(scatter, ax=axes[0])
cbar.set_label('Gender (0=Female, 1=Male)', fontsize=11)

# Plot 2: Residuals (Error Distribution)
residuals = y_true_age - y_pred_age
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[1].set_xlabel('Prediction Error (months)', fontsize=13)
axes[1].set_ylabel('Frequency', fontsize=13)
axes[1].set_title(f'Error Distribution\nMean: {np.mean(residuals):.2f}, Std: {np.std(residuals):.2f}', 
                 fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('regression_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Regression analysis plot saved as 'regression_analysis.png'")

In [ ]:
# ============================================================================
# VISUALIZATION: CLASSIFICATION ANALYSIS
# ============================================================================
print("\n>>> PLOTTING CLASSIFICATION RESULTS")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true_class, y_pred_class, 
                          target_names=CLASS_NAMES, 
                          digits=3))

# Confusion Matrix
cm = confusion_matrix(y_true_class, y_pred_class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES,
            ax=axes[0], cbar_kws={'label': 'Count'})
axes[0].set_xlabel('Predicted Class', fontsize=13)
axes[0].set_ylabel('True Class', fontsize=13)
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')

# Plot 2: Normalized
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Greens',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES,
            ax=axes[1], cbar_kws={'label': 'Proportion'})
axes[1].set_xlabel('Predicted Class', fontsize=13)
axes[1].set_ylabel('True Class', fontsize=13)
axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('classification_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Classification confusion matrix saved as 'classification_confusion_matrix.png'")

In [ ]:
# ============================================================================
# GENDER BIAS ANALYSIS
# ============================================================================
print("\n>>> ANALYZING GENDER BIAS")

# Create analysis dataframe
analysis_df = pd.DataFrame({
    'True_Age': y_true_age,
    'Pred_Age': y_pred_age,
    'Error': y_true_age - y_pred_age,
    'Abs_Error': np.abs(y_true_age - y_pred_age),
    'Gender': test_genders_all.numpy().flatten(),
    'True_Class': y_true_class,
    'Pred_Class': y_pred_class
})

# Calculate metrics by gender
male_mae = analysis_df[analysis_df['Gender'] == 1.0]['Abs_Error'].mean()
female_mae = analysis_df[analysis_df['Gender'] == 0.0]['Abs_Error'].mean()
male_count = (analysis_df['Gender'] == 1.0).sum()
female_count = (analysis_df['Gender'] == 0.0).sum()

print(f"\nRegression Error by Gender:")
print(f"  Male (n={male_count}): MAE = {male_mae:.2f} months")
print(f"  Female (n={female_count}): MAE = {female_mae:.2f} months")
print(f"  Difference: {abs(male_mae - female_mae):.2f} months")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: MAE by Gender
gender_labels = ['Male', 'Female']
mae_values = [male_mae, female_mae]
colors = ['#3498db', '#e74c3c']
bars = axes[0].bar(gender_labels, mae_values, color=colors, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Mean Absolute Error (months)', fontsize=13)
axes[0].set_title('Regression Error by Gender', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')
# Add value labels on bars
for bar, val in zip(bars, mae_values):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.2f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

# Plot 2: Error Distribution by Gender
male_errors = analysis_df[analysis_df['Gender'] == 1.0]['Error']
female_errors = analysis_df[analysis_df['Gender'] == 0.0]['Error']
axes[1].hist(male_errors, bins=40, alpha=0.6, label='Male', color='#3498db', edgecolor='black')
axes[1].hist(female_errors, bins=40, alpha=0.6, label='Female', color='#e74c3c', edgecolor='black')
axes[1].axvline(0, color='black', linestyle='--', linewidth=2)
axes[1].set_xlabel('Prediction Error (months)', fontsize=13)
axes[1].set_ylabel('Frequency', fontsize=13)
axes[1].set_title('Error Distribution by Gender', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3, axis='y')

# Plot 3: Classification Accuracy by Gender
male_acc = (analysis_df[analysis_df['Gender'] == 1.0]['True_Class'] == 
            analysis_df[analysis_df['Gender'] == 1.0]['Pred_Class']).mean()
female_acc = (analysis_df[analysis_df['Gender'] == 0.0]['True_Class'] == 
              analysis_df[analysis_df['Gender'] == 0.0]['Pred_Class']).mean()
acc_values = [male_acc * 100, female_acc * 100]
bars = axes[2].bar(gender_labels, acc_values, color=colors, edgecolor='black', linewidth=1.5)
axes[2].set_ylabel('Classification Accuracy (%)', fontsize=13)
axes[2].set_title('Classification Accuracy by Gender', fontsize=14, fontweight='bold')
axes[2].set_ylim([0, 100])
axes[2].grid(True, alpha=0.3, axis='y')
# Add value labels on bars
for bar, val in zip(bars, acc_values):
    height = bar.get_height()
    axes[2].text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.1f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('gender_bias_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Gender bias analysis plot saved as 'gender_bias_analysis.png'")

In [ ]:
# ============================================================================
# ERROR ANALYSIS BY AGE GROUP
# ============================================================================
print("\n>>> ERROR ANALYSIS BY AGE GROUP")

# Calculate MAE for each age class
class_mae = []
class_counts = []
for i in range(NUM_CLASSES):
    mask = analysis_df['True_Class'] == i
    if mask.sum() > 0:
        mae = analysis_df[mask]['Abs_Error'].mean()
        count = mask.sum()
        class_mae.append(mae)
        class_counts.append(count)
        print(f"  {CLASS_NAMES[i]}: MAE = {mae:.2f} months (n={count})")
    else:
        class_mae.append(0)
        class_counts.append(0)

# Visualization
fig, ax = plt.subplots(figsize=(12, 6))
x_pos = np.arange(len(CLASS_NAMES))
bars = ax.bar(x_pos, class_mae, color='teal', edgecolor='black', linewidth=1.5, alpha=0.7)
ax.set_xticks(x_pos)
ax.set_xticklabels(CLASS_NAMES, fontsize=12)
ax.set_ylabel('Mean Absolute Error (months)', fontsize=13)
ax.set_title('Regression Error by Age Group', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Add value labels and sample counts
for i, (bar, mae, count) in enumerate(zip(bars, class_mae, class_counts)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{mae:.2f}\n(n={count})', 
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('error_by_age_group.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Age group error analysis plot saved as 'error_by_age_group.png'")

In [ ]:



# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

print(f"\n📊 DATASET:")
print(f"  Total samples: {len(df)}")
print(f"  Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

print(f"\n🏗️ MODEL:")
print(f"  Architecture: InceptionV3 + Multi-Task Heads")
print(f"  Parameters: {total_params:,}")
print(f"  Input Size: {IMG_SIZE}x{IMG_SIZE}")

print(f"\n⚙️ TRAINING:")
print(f"  Strategy: {strategy.__class__.__name__}")
print(f"  GPUs Used: {strategy.num_replicas_in_sync}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Epochs Trained: {len(history.history['loss'])}")

print(f"\n📈 TEST PERFORMANCE:")
print(f"  Regression MAE: {test_results[3]:.2f} months")
print(f"  Regression RMSE: {np.sqrt(test_results[4]):.2f} months")
print(f"  Classification Accuracy: {test_results[5]*100:.2f}%")

print(f"\n👥 GENDER ANALYSIS:")
print(f"  Male MAE: {male_mae:.2f} months")
print(f"  Female MAE: {female_mae:.2f} months")
print(f"  Bias: {abs(male_mae - female_mae):.2f} months")

print(f"\n💾 SAVED FILES:")
print(f"  ✓ best_bone_age_model.keras")
print(f"  ✓ training_history.csv")
print(f"  ✓ training_history.png")
print(f"  ✓ regression_analysis.png")
print(f"  ✓ classification_confusion_matrix.png")
print(f"  ✓ gender_bias_analysis.png")
print(f"  ✓ error_by_age_group.png")

print("\n" + "="*80)
print("✅ PROJECT COMPLETE!")
print("="*80)

In [ ]:
# ============================================================================
# COMPREHENSIVE MODEL EVALUATION CODE
# Add this to your Kaggle notebook after training completes
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    classification_report, confusion_matrix,
    precision_recall_fscore_support
)
from scipy import stats

print("="*80)
print("COMPREHENSIVE MODEL EVALUATION")
print("="*80)

# ============================================================================
# 1. OVERFITTING ANALYSIS
# ============================================================================
print("\n>>> OVERFITTING ANALYSIS")

# Load training history
history_df = pd.read_csv('training_history.csv')
final_epoch = history_df.iloc[-1]

# Calculate gaps
train_mae = final_epoch['age_out_mae']
val_mae = final_epoch['val_age_out_mae']
mae_gap_pct = ((val_mae - train_mae) / train_mae) * 100

train_acc = final_epoch['class_out_accuracy']
val_acc = final_epoch['val_class_out_accuracy']
acc_gap_pct = ((train_acc - val_acc) / train_acc) * 100

print(f"\nTrain-Validation Gaps:")
print(f"  MAE Gap: {abs(mae_gap_pct):.2f}%")
print(f"  Accuracy Gap: {abs(acc_gap_pct):.2f}%")

# Assessment
if abs(mae_gap_pct) < 5:
    print(f"  MAE Assessment: ✅ EXCELLENT - No overfitting")
elif abs(mae_gap_pct) < 10:
    print(f"  MAE Assessment: ✅ GOOD - Minimal overfitting")
elif abs(mae_gap_pct) < 20:
    print(f"  MAE Assessment: ⚠️ MODERATE - Some overfitting")
else:
    print(f"  MAE Assessment: ❌ HIGH - Significant overfitting")

# ============================================================================
# 2. COMPREHENSIVE REGRESSION METRICS
# ============================================================================
print("\n>>> REGRESSION METRICS (Age Prediction)")

# Assuming you have: y_true_age, y_pred_age from your training notebook

# Calculate metrics
mae = mean_absolute_error(y_true_age, y_pred_age)
mse = mean_squared_error(y_true_age, y_pred_age)
rmse = np.sqrt(mse)
r2 = r2_score(y_true_age, y_pred_age)

# Calculate MAPE manually (avoid division by zero)
mape = np.mean(np.abs((y_true_age - y_pred_age) / np.maximum(y_true_age, 1))) * 100

# Calculate median absolute error
median_ae = np.median(np.abs(y_true_age - y_pred_age))

# Calculate max error
max_error = np.max(np.abs(y_true_age - y_pred_age))

print(f"\nRegression Performance:")
print(f"  MAE: {mae:.2f} months")
print(f"  RMSE: {rmse:.2f} months")
print(f"  R² Score: {r2:.4f}")
print(f"  MAPE: {mape:.2f}%")
print(f"  Median AE: {median_ae:.2f} months")
print(f"  Max Error: {max_error:.2f} months")

# Interpretation
print(f"\nInterpretation:")
if mae < 8:
    print(f"  ✅ EXCELLENT: MAE < 8 months (medical-grade)")
elif mae < 10:
    print(f"  ✅ GOOD: MAE < 10 months (acceptable)")
elif mae < 15:
    print(f"  ⚠️ FAIR: MAE < 15 months (needs improvement)")
else:
    print(f"  ❌ POOR: MAE > 15 months (significant improvement needed)")

if r2 > 0.9:
    print(f"  ✅ EXCELLENT: R² > 0.9 (very strong correlation)")
elif r2 > 0.8:
    print(f"  ✅ GOOD: R² > 0.8 (strong correlation)")
elif r2 > 0.7:
    print(f"  ⚠️ FAIR: R² > 0.7 (moderate correlation)")
else:
    print(f"  ❌ POOR: R² < 0.7 (weak correlation)")

# ============================================================================
# 3. COMPREHENSIVE CLASSIFICATION METRICS
# ============================================================================
print("\n>>> CLASSIFICATION METRICS (Age Category)")

# Assuming you have: y_true_class, y_pred_class

# Calculate metrics
accuracy = np.mean(y_true_class == y_pred_class)
precision, recall, f1, support = precision_recall_fscore_support(
    y_true_class, y_pred_class, average='weighted'
)

print(f"\nClassification Performance:")
print(f"  Accuracy: {accuracy*100:.2f}%")
print(f"  Precision (weighted): {precision:.4f}")
print(f"  Recall (weighted): {recall:.4f}")
print(f"  F1-Score (weighted): {f1:.4f}")

# Per-class metrics
print(f"\nPer-Class Performance:")
class_names = ['Infant (0-2y)', 'Pre-Puberty (2-10y)', 'Puberty (10-16y)', 'Young Adult (16+y)']
for i, class_name in enumerate(class_names):
    mask = y_true_class == i
    if mask.sum() > 0:
        class_acc = np.mean(y_pred_class[mask] == i)
        print(f"  {class_name}: {class_acc*100:.2f}% (n={mask.sum()})")

# ============================================================================
# 4. RESIDUAL ANALYSIS
# ============================================================================
print("\n>>> RESIDUAL ANALYSIS")

residuals = y_true_age - y_pred_age

# Statistical tests
print(f"\nResidual Statistics:")
print(f"  Mean: {np.mean(residuals):.4f} (should be ~0)")
print(f"  Std Dev: {np.std(residuals):.4f}")
print(f"  Skewness: {stats.skew(residuals):.4f} (should be ~0)")
print(f"  Kurtosis: {stats.kurtosis(residuals):.4f} (should be ~0)")

# Normality test
statistic, p_value = stats.shapiro(residuals[:5000])  # Shapiro-Wilk test (max 5000 samples)
print(f"\nNormality Test (Shapiro-Wilk):")
print(f"  p-value: {p_value:.4f}")
if p_value > 0.05:
    print(f"  ✅ Residuals are normally distributed (good!)")
else:
    print(f"  ⚠️ Residuals are not perfectly normal (acceptable for large datasets)")

# ============================================================================
# 5. PREDICTION CONFIDENCE ANALYSIS
# ============================================================================
print("\n>>> PREDICTION CONFIDENCE ANALYSIS")

# Calculate prediction intervals
errors = np.abs(residuals)
percentiles = [50, 75, 90, 95, 99]

print(f"\nPrediction Confidence Intervals:")
for p in percentiles:
    interval = np.percentile(errors, p)
    print(f"  {p}% of predictions within ±{interval:.2f} months")

# ============================================================================
# 6. GENERALIZATION CHECK
# ============================================================================
print("\n>>> GENERALIZATION CHECK")

# Compare test vs validation performance
test_mae = mae  # From above
val_mae_final = final_epoch['val_age_out_mae']
generalization_gap = abs(test_mae - val_mae_final)
generalization_gap_pct = (generalization_gap / val_mae_final) * 100

print(f"\nTest vs Validation:")
print(f"  Validation MAE: {val_mae_final:.2f} months")
print(f"  Test MAE: {test_mae:.2f} months")
print(f"  Gap: {generalization_gap:.2f} months ({generalization_gap_pct:.2f}%)")

if generalization_gap_pct < 5:
    print(f"  ✅ EXCELLENT: Model generalizes very well")
elif generalization_gap_pct < 10:
    print(f"  ✅ GOOD: Model generalizes well")
else:
    print(f"  ⚠️ WARNING: Significant generalization gap")

# ============================================================================
# 7. FINAL ASSESSMENT
# ============================================================================
print("\n" + "="*80)
print("FINAL MODEL ASSESSMENT")
print("="*80)

# Overall score (0-100)
score = 0

# Regression performance (40 points)
if mae < 8:
    score += 40
elif mae < 10:
    score += 35
elif mae < 12:
    score += 30
else:
    score += 20

# Classification performance (30 points)
if accuracy > 0.9:
    score += 30
elif accuracy > 0.85:
    score += 25
elif accuracy > 0.8:
    score += 20
else:
    score += 15

# Overfitting (20 points)
if abs(mae_gap_pct) < 5:
    score += 20
elif abs(mae_gap_pct) < 10:
    score += 15
elif abs(mae_gap_pct) < 20:
    score += 10
else:
    score += 5

# Generalization (10 points)
if generalization_gap_pct < 5:
    score += 10
elif generalization_gap_pct < 10:
    score += 7
else:
    score += 5

print(f"\n🎯 OVERALL MODEL SCORE: {score}/100")

if score >= 90:
    print(f"   Grade: A+ (EXCELLENT - Production Ready)")
elif score >= 80:
    print(f"   Grade: A (VERY GOOD - Production Ready)")
elif score >= 70:
    print(f"   Grade: B (GOOD - Minor improvements recommended)")
elif score >= 60:
    print(f"   Grade: C (FAIR - Improvements needed)")
else:
    print(f"   Grade: D (POOR - Significant improvements needed)")

print("\n" + "="*80)
print("EVALUATION COMPLETE")
print("="*80)

# ============================================================================
# 8. VISUALIZATION: COMPREHENSIVE EVALUATION PLOTS
# ============================================================================
print("\n>>> GENERATING COMPREHENSIVE EVALUATION PLOTS")

fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

# Plot 1: Residual Plot
ax1 = fig.add_subplot(gs[0, 0])
ax1.scatter(y_pred_age, residuals, alpha=0.5, s=20)
ax1.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax1.set_xlabel('Predicted Age (months)')
ax1.set_ylabel('Residuals (months)')
ax1.set_title('Residual Plot', fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: Residual Distribution
ax2 = fig.add_subplot(gs[0, 1])
ax2.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
ax2.axvline(x=0, color='r', linestyle='--', linewidth=2)
ax2.set_xlabel('Residuals (months)')
ax2.set_ylabel('Frequency')
ax2.set_title('Residual Distribution', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Q-Q Plot
ax3 = fig.add_subplot(gs[0, 2])
stats.probplot(residuals, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot (Normality Check)', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Plot 4: Error by Predicted Value
ax4 = fig.add_subplot(gs[0, 3])
ax4.scatter(y_pred_age, np.abs(residuals), alpha=0.5, s=20)
ax4.set_xlabel('Predicted Age (months)')
ax4.set_ylabel('Absolute Error (months)')
ax4.set_title('Error vs Predicted Value', fontweight='bold')
ax4.grid(True, alpha=0.3)

# Plot 5: Learning Curves
ax5 = fig.add_subplot(gs[1, :2])
ax5.plot(history_df['age_out_mae'], label='Train MAE', linewidth=2)
ax5.plot(history_df['val_age_out_mae'], label='Val MAE', linewidth=2)
ax5.fill_between(range(len(history_df)), 
                 history_df['age_out_mae'], 
                 history_df['val_age_out_mae'], 
                 alpha=0.2)
ax5.set_xlabel('Epoch')
ax5.set_ylabel('MAE (months)')
ax5.set_title('Learning Curves (Overfitting Check)', fontweight='bold')
ax5.legend()
ax5.grid(True, alpha=0.3)

# Plot 6: Overfitting Gap
ax6 = fig.add_subplot(gs[1, 2:])
gap_series = history_df['val_age_out_mae'] - history_df['age_out_mae']
ax6.plot(gap_series, linewidth=2, color='purple')
ax6.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax6.fill_between(range(len(history_df)), 0, gap_series, 
                 where=(gap_series > 0), alpha=0.3, color='red')
ax6.set_xlabel('Epoch')
ax6.set_ylabel('Val MAE - Train MAE')
ax6.set_title('Overfitting Gap Over Time', fontweight='bold')
ax6.grid(True, alpha=0.3)

# Plot 7: Error Distribution by Age Group
ax7 = fig.add_subplot(gs[2, :2])
for i, class_name in enumerate(class_names):
    mask = y_true_class == i
    if mask.sum() > 0:
        class_errors = np.abs(residuals[mask])
        ax7.hist(class_errors, bins=30, alpha=0.5, label=class_name, edgecolor='black')
ax7.set_xlabel('Absolute Error (months)')
ax7.set_ylabel('Frequency')
ax7.set_title('Error Distribution by Age Group', fontweight='bold')
ax7.legend()
ax7.grid(True, alpha=0.3, axis='y')

# Plot 8: Confusion Matrix
ax8 = fig.add_subplot(gs[2, 2:])
cm = confusion_matrix(y_true_class, y_pred_class)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names, ax=ax8)
ax8.set_xlabel('Predicted Class')
ax8.set_ylabel('True Class')
ax8.set_title('Confusion Matrix', fontweight='bold')

plt.suptitle('Comprehensive Model Evaluation Dashboard', fontsize=16, fontweight='bold', y=0.995)
plt.savefig('comprehensive_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Comprehensive evaluation plot saved as 'comprehensive_evaluation.png'")
print("\n✅ EVALUATION COMPLETE!")


In [ ]:
# ============================================================================
# ENHANCEMENT 1: CALIBRATION & UNCERTAINTY ESTIMATION
# Time: 2 hours | Impact: MAE 9.22 → 8.8 months
# ============================================================================

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.isotonic import IsotonicRegression
import matplotlib.pyplot as plt
import seaborn as sns

print("="*80)
print("ENHANCEMENT 1: CALIBRATION & UNCERTAINTY ESTIMATION")
print("="*80)

# ============================================================================
# PART 1: BIAS CALIBRATION
# ============================================================================
print("\n>>> PART 1: BIAS CALIBRATION")

# From your evaluation, residual mean = 1.47 months
BIAS_CORRECTION = 1.47

# Simple calibration
y_pred_calibrated = y_pred_age + BIAS_CORRECTION

# Evaluate improvement
mae_before = np.mean(np.abs(y_true_age - y_pred_age))
mae_after = np.mean(np.abs(y_true_age - y_pred_calibrated))

print(f"\nSimple Bias Correction:")
print(f"  MAE before: {mae_before:.2f} months")
print(f"  MAE after: {mae_after:.2f} months")
print(f"  Improvement: {mae_before - mae_after:.2f} months")

# ============================================================================
# PART 2: ISOTONIC REGRESSION CALIBRATION (Advanced)
# ============================================================================
print("\n>>> PART 2: ISOTONIC REGRESSION CALIBRATION")

# Fit calibrator on validation set (you need val predictions)
# If you don't have val predictions, use a subset of test set for calibration
# and evaluate on the rest

# Split test set for calibration
n_cal = len(y_true_age) // 2
y_true_cal = y_true_age[:n_cal]
y_pred_cal = y_pred_age[:n_cal]
y_true_eval = y_true_age[n_cal:]
y_pred_eval = y_pred_age[n_cal:]

# Fit isotonic regression
calibrator = IsotonicRegression(out_of_bounds='clip')
calibrator.fit(y_pred_cal, y_true_cal)

# Apply calibration
y_pred_isotonic = calibrator.predict(y_pred_eval)

# Evaluate
mae_isotonic = np.mean(np.abs(y_true_eval - y_pred_isotonic))
mae_uncalibrated = np.mean(np.abs(y_true_eval - y_pred_eval))

print(f"\nIsotonic Regression Calibration:")
print(f"  MAE before: {mae_uncalibrated:.2f} months")
print(f"  MAE after: {mae_isotonic:.2f} months")
print(f"  Improvement: {mae_uncalibrated - mae_isotonic:.2f} months")

# Visualize calibration curve
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(y_pred_cal, y_true_cal, alpha=0.3, s=20, label='Calibration data')
plt.plot([0, 230], [0, 230], 'k--', lw=2, label='Perfect calibration')
x_range = np.linspace(y_pred_cal.min(), y_pred_cal.max(), 100)
plt.plot(x_range, calibrator.predict(x_range), 'r-', lw=2, label='Calibration curve')
plt.xlabel('Predicted Age (months)')
plt.ylabel('True Age (months)')
plt.title('Calibration Curve')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
residuals_before = y_true_eval - y_pred_eval
residuals_after = y_true_eval - y_pred_isotonic
plt.hist(residuals_before, bins=50, alpha=0.5, label='Before calibration', edgecolor='black')
plt.hist(residuals_after, bins=50, alpha=0.5, label='After calibration', edgecolor='black')
plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
plt.xlabel('Residuals (months)')
plt.ylabel('Frequency')
plt.title('Residual Distribution')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('calibration_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Calibration plot saved as 'calibration_results.png'")

# ============================================================================
# PART 3: UNCERTAINTY ESTIMATION (Monte Carlo Dropout)
# ============================================================================
print("\n>>> PART 3: UNCERTAINTY ESTIMATION")

def predict_with_uncertainty(model, images, genders, n_iterations=100):
    """
    Use Monte Carlo Dropout to estimate prediction uncertainty
    
    Args:
        model: Trained Keras model
        images: Input images
        genders: Gender inputs
        n_iterations: Number of forward passes
    
    Returns:
        mean_predictions: Mean predictions
        std_predictions: Standard deviation (uncertainty)
    """
    predictions = []
    
    # Enable dropout at inference time
    for layer in model.layers:
        if 'dropout' in layer.name.lower():
            layer.training = True
    
    # Multiple forward passes with dropout
    print(f"Running {n_iterations} forward passes...")
    for i in range(n_iterations):
        if i % 20 == 0:
            print(f"  Iteration {i}/{n_iterations}")
        
        pred = model.predict([images, genders], verbose=0)
        predictions.append(pred[0])  # Age predictions
    
    # Reset dropout
    for layer in model.layers:
        if 'dropout' in layer.name.lower():
            layer.training = False
    
    predictions = np.array(predictions)
    
    # Calculate statistics
    mean_pred = predictions.mean(axis=0).flatten()
    std_pred = predictions.std(axis=0).flatten()
    
    return mean_pred, std_pred

# Apply to test set (use a subset for speed)
n_samples = min(500, len(test_images_all))
print(f"\nApplying uncertainty estimation to {n_samples} samples...")

y_pred_mean, y_pred_std = predict_with_uncertainty(
    model, 
    test_images_all[:n_samples], 
    test_genders_all[:n_samples],
    n_iterations=50  # Reduce for speed
)

# Analyze uncertainty
print(f"\n>>> UNCERTAINTY ANALYSIS")
print(f"Mean uncertainty: {y_pred_std.mean():.2f} months")
print(f"Median uncertainty: {np.median(y_pred_std):.2f} months")
print(f"Max uncertainty: {y_pred_std.max():.2f} months")

# Identify uncertain predictions
uncertainty_threshold = 15  # months
uncertain_mask = y_pred_std > uncertainty_threshold
n_uncertain = uncertain_mask.sum()

print(f"\nUncertain predictions (std > {uncertainty_threshold} months): {n_uncertain} ({n_uncertain/len(y_pred_std)*100:.1f}%)")

# Correlation between uncertainty and error
y_true_subset = y_true_age[:n_samples]
errors = np.abs(y_true_subset - y_pred_mean)
correlation = np.corrcoef(y_pred_std, errors)[0, 1]

print(f"\nCorrelation between uncertainty and error: {correlation:.3f}")
if correlation > 0.3:
    print("✅ Good! High uncertainty correlates with high error")
else:
    print("⚠️ Uncertainty may need calibration")

# Visualize uncertainty
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Uncertainty distribution
axes[0, 0].hist(y_pred_std, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=uncertainty_threshold, color='r', linestyle='--', linewidth=2, label=f'Threshold ({uncertainty_threshold} months)')
axes[0, 0].set_xlabel('Prediction Uncertainty (std, months)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Prediction Uncertainty')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Plot 2: Uncertainty vs Error
axes[0, 1].scatter(y_pred_std, errors, alpha=0.5, s=30)
axes[0, 1].set_xlabel('Prediction Uncertainty (std, months)')
axes[0, 1].set_ylabel('Absolute Error (months)')
axes[0, 1].set_title(f'Uncertainty vs Error (correlation: {correlation:.3f})')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Predictions with uncertainty bands
sorted_idx = np.argsort(y_pred_mean)
axes[1, 0].plot(y_true_subset[sorted_idx], 'b-', label='True Age', linewidth=2)
axes[1, 0].plot(y_pred_mean[sorted_idx], 'r-', label='Predicted Age', linewidth=2)
axes[1, 0].fill_between(
    range(len(sorted_idx)),
    (y_pred_mean - 2*y_pred_std)[sorted_idx],
    (y_pred_mean + 2*y_pred_std)[sorted_idx],
    alpha=0.3, color='red', label='95% Confidence'
)
axes[1, 0].set_xlabel('Sample (sorted by prediction)')
axes[1, 0].set_ylabel('Age (months)')
axes[1, 0].set_title('Predictions with Uncertainty Bands')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Error by uncertainty quartile
quartiles = np.percentile(y_pred_std, [25, 50, 75])
q1_mask = y_pred_std <= quartiles[0]
q2_mask = (y_pred_std > quartiles[0]) & (y_pred_std <= quartiles[1])
q3_mask = (y_pred_std > quartiles[1]) & (y_pred_std <= quartiles[2])
q4_mask = y_pred_std > quartiles[2]

mae_by_quartile = [
    errors[q1_mask].mean(),
    errors[q2_mask].mean(),
    errors[q3_mask].mean(),
    errors[q4_mask].mean()
]

axes[1, 1].bar(['Q1\n(Low)', 'Q2', 'Q3', 'Q4\n(High)'], mae_by_quartile, 
               color=['green', 'yellow', 'orange', 'red'], edgecolor='black', linewidth=1.5)
axes[1, 1].set_ylabel('Mean Absolute Error (months)')
axes[1, 1].set_xlabel('Uncertainty Quartile')
axes[1, 1].set_title('Error by Uncertainty Level')
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Add values on bars
for i, v in enumerate(mae_by_quartile):
    axes[1, 1].text(i, v + 0.5, f'{v:.1f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('uncertainty_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Uncertainty analysis plot saved as 'uncertainty_analysis.png'")

# ============================================================================
# PART 4: PRACTICAL USAGE - FLAG UNCERTAIN CASES
# ============================================================================
print("\n>>> PART 4: PRACTICAL USAGE")

# Create decision framework
def make_prediction_with_confidence(model, image, gender, threshold=15):
    """
    Make prediction with confidence assessment
    
    Returns:
        prediction: Mean prediction
        uncertainty: Standard deviation
        confidence: 'high', 'medium', or 'low'
        recommendation: Action to take
    """
    mean_pred, std_pred = predict_with_uncertainty(
        model, image[None, ...], gender[None, ...], n_iterations=50
    )
    
    # Assess confidence
    if std_pred[0] < 10:
        confidence = 'high'
        recommendation = 'Accept prediction'
    elif std_pred[0] < threshold:
        confidence = 'medium'
        recommendation = 'Review if critical'
    else:
        confidence = 'low'
        recommendation = 'Manual review recommended'
    
    return mean_pred[0], std_pred[0], confidence, recommendation

# Example usage
print("\nExample predictions with confidence:")
for i in range(min(5, n_samples)):
    pred, unc, conf, rec = make_prediction_with_confidence(
        model, test_images_all[i], test_genders_all[i]
    )
    true_age = y_true_age[i]
    error = abs(true_age - pred)
    
    print(f"\nSample {i+1}:")
    print(f"  True age: {true_age:.1f} months")
    print(f"  Predicted: {pred:.1f} ± {unc:.1f} months")
    print(f"  Error: {error:.1f} months")
    print(f"  Confidence: {conf}")
    print(f"  Recommendation: {rec}")

# ============================================================================
# SUMMARY
# ============================================================================
print("\n" + "="*80)
print("SUMMARY - CALIBRATION & UNCERTAINTY")
print("="*80)

print(f"\n✅ CALIBRATION RESULTS:")
print(f"  Simple bias correction: {mae_before:.2f} → {mae_after:.2f} months")
print(f"  Isotonic calibration: {mae_uncalibrated:.2f} → {mae_isotonic:.2f} months")
print(f"  Best improvement: {max(mae_before - mae_after, mae_uncalibrated - mae_isotonic):.2f} months")

print(f"\n✅ UNCERTAINTY ESTIMATION:")
print(f"  Mean uncertainty: {y_pred_std.mean():.2f} months")
print(f"  Uncertain cases: {n_uncertain} ({n_uncertain/len(y_pred_std)*100:.1f}%)")
print(f"  Uncertainty-error correlation: {correlation:.3f}")

print(f"\n✅ PRACTICAL IMPACT:")
print(f"  Can identify {n_uncertain} cases needing manual review")
print(f"  High-confidence predictions have {mae_by_quartile[0]:.1f} months MAE")
print(f"  Low-confidence predictions have {mae_by_quartile[3]:.1f} months MAE")

print(f"\n✅ FILES SAVED:")
print(f"  - calibration_results.png")
print(f"  - uncertainty_analysis.png")

print("\n" + "="*80)
print("✅ ENHANCEMENT 1 COMPLETE!")
print("="*80)


In [ ]:
# ============================================================================
# SIMPLE: Package all your files into one ZIP
# ============================================================================

import zipfile
import os

# Create ZIP with all your actual files
with zipfile.ZipFile('/kaggle/working/MY_BONE_AGE_FILES.zip', 'w') as zipf:
    for file in os.listdir('/kaggle/working'):
        if not file.endswith('.zip'):  # Don't include existing ZIPs
            zipf.write(f'/kaggle/working/{file}', file)
            print(f"✓ Added: {file}")

print("\n✅ Created: MY_BONE_AGE_FILES.zip")
print("\nYour files:")
print("  • best_bone_age_model.keras")
print("  • training_history.png")
print("  • classification_confusion_matrix.png")
print("  • error_by_age_group.png")
print("  • gender_bias_analysis.png")
print("  • regression_analysis.png")
print("  • training_history.csv")

print("\n📥 TO DOWNLOAD:")
print("1. Click 'Save Version' → 'Save & Run All'")
print("2. Wait for completion")
print("3. Click 'Output' tab")
print("4. Download MY_BONE_AGE_FILES.zip")
